In [1]:
from pathlib import Path
import os
import shutil
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import pandas as pd

In [3]:
#@jit(nopython=True)
def transform_np_allpoints(point, center, scale, resolution=256, invert=False):
    """Generate and affine transformation matrix.

    Given a set of points, a center, a scale and a targer resolution, the
    function generates and affine transformation matrix. If invert is ``True``
    it will produce the inverse transformation.

    Arguments:
        point {numpy.array} -- the input 2D point
        center {numpy.array} -- the center around which to perform the transformations
        scale {float} -- the scale of the face/object
        resolution {float} -- the output resolution

    Keyword Arguments:
        invert {bool} -- define wherever the function should produce the direct or the
        inverse transformation matrix (default: {False})
    """

    h = 200.0 * scale
    t = np.eye(3).reshape(-1,1)
    t[0] = resolution / h
    t[4] = resolution / h
    t[2] = resolution * (-center[0] / h + 0.5)
    t[5] = resolution * (-center[1] / h + 0.5)
    t = t.reshape(3,3)
    
    if invert:
        t = np.ascontiguousarray(np.linalg.pinv(t))

    new_point = t.dot(np.column_stack((point, np.ones((len(point),1)))).T)[0:2,:].T

    return new_point


#@jit(nopython=True)
def transform_np(point, center, scale, resolution=256, invert=False):
    """Generate and affine transformation matrix.
    Given a set of points, a center, a scale and a targer resolution, the
    function generates and affine transformation matrix. If invert is ``True``
    it will produce the inverse transformation.
    Arguments:
        point {numpy.array} -- the input 2D point
        center {numpy.array} -- the center around which to perform the transformations
        scale {float} -- the scale of the face/object
        resolution {float} -- the output resolution
    Keyword Arguments:
        invert {bool} -- define wherever the function should produce the direct or the
        inverse transformation matrix (default: {False})
    """
    _pt = np.ones(3)
    _pt[0] = point[0]
    _pt[1] = point[1]
    
    h = 200.0 * scale
    t = np.eye(3).reshape(-1,1)
    t[0] = resolution / h
    t[4] = resolution / h
    t[2] = resolution * (-center[0] / h + 0.5)
    t[5] = resolution * (-center[1] / h + 0.5)
    t = t.reshape(3,3)
    
    if invert:
        t = np.ascontiguousarray(np.linalg.pinv(t))
    
    new_point = np.dot(t, _pt)[0:2]
    
    return new_point.astype(np.int32)

def crop(image, center, scale, resolution=256.0):
    """Center crops an image or set of heatmaps
    Arguments:
        image {numpy.array} -- an rgb image
        center {numpy.array} -- the center of the object, usually the same as of the bounding box
        scale {float} -- scale of the face
    Keyword Arguments:
        resolution {float} -- the size of the output cropped image (default: {256.0})
    Returns:
        [type] -- [description]
    """  # Crop around the center point
    """ Crops the image around the center. Input is expected to be an np.ndarray """
    ul = transform_np(np.array([1, 1], dtype=int), center, scale, resolution, True)
    br = transform_np(np.array([resolution, resolution], dtype=int), center, scale, resolution, True)
    # pad = math.ceil(torch.norm((ul - br).float()) / 2.0 - (br[0] - ul[0]) / 2.0)
    if image.ndim > 2:
        newDim = np.array([br[1] - ul[1], br[0] - ul[0],
                           image.shape[2]], dtype=np.int32)
        newImg = np.zeros(newDim, dtype=np.uint8)
    else:
        newDim = np.array([br[1] - ul[1], br[0] - ul[0]], dtype=np.int)
        newImg = np.zeros(newDim, dtype=np.uint8)
    ht = image.shape[0]
    wd = image.shape[1]
    newX = np.array(
        [max(1, -ul[0] + 1), min(br[0], wd) - ul[0]], dtype=np.int32)
    newY = np.array(
        [max(1, -ul[1] + 1), min(br[1], ht) - ul[1]], dtype=np.int32)
    oldX = np.array([max(1, ul[0] + 1), min(br[0], wd)], dtype=np.int32)
    oldY = np.array([max(1, ul[1] + 1), min(br[1], ht)], dtype=np.int32)
    newImg[newY[0] - 1:newY[1], newX[0] - 1:newX[1]
           ] = image[oldY[0] - 1:oldY[1], oldX[0] - 1:oldX[1], :]

    newImg = Image.fromarray(newImg).resize((int(resolution), int(resolution)),resample=Image.LINEAR)

    return newImg

In [8]:
size = (256,256)
dfs_name = [r'../300w/face_landmarks_300w_train.csv', 
            r'../300w/face_landmarks_300w_valid_challenge.csv',
            r'../300w/face_landmarks_300w_valid_common.csv',
            r'../300w/face_landmarks_300w_test.csv']

dfs_name_save = [r'../300w/cropped/face_landmarks_300w_train_cropped.csv', 
                 r'../300w/cropped/face_landmarks_300w_valid_challenge_cropped.csv',
                 r'../300w/cropped/face_landmarks_300w_valid_common_cropped.csv', 
                 r'../300w/cropped/face_landmarks_300w_test_cropped.csv']

new_cols = ['image_name']
for i in range(68):
    new_cols.append(f'landmark_{i}_x')
    new_cols.append(f'landmark_{i}_y')

    
    
reference_scale=195
for k, df_name in enumerate(dfs_name):

        
    df = pd.read_csv(df_name, index_col=None)
    data = []
    
    for row in df.iterrows():
        name = row[1][0]

        new_path = Path('../300w/cropped')/name
        
        if not new_path.is_file():
            
            points = row[1][4:].to_numpy(dtype=float).reshape(-1,2)
            
            x1,x2,y1,y2 = points[:,0].min(), points[:,0].max(),points[:,1].min(), points[:,1].max()
            center = np.array([x2 - (x2 - x1) / 2.0, y2 - (y2 - y1) / 2.0], dtype=float)
            center[1] = center[1] - (y2 - y1) * 0.12
            scale = (x2 - x1 + y2 - y1) / reference_scale

            im = np.asarray(Image.open(Path('../300w/images/')/name).convert('RGB'))
            img_crop = crop(im,center,scale,256)
            points_crop = transform_np_allpoints(points, center,scale, 256)

            new_path_parents = new_path.parent
            if not new_path_parents.exists():
                new_path_parents.mkdir(parents=True)

            img_crop.save(new_path)


            datum = []
            datum = [name]
            for x,y in points_crop:
                datum.append(x)
                datum.append(y)


            data.append(datum)



    df_new = pd.DataFrame(data, columns=new_cols)
    df_new.to_csv(dfs_name_save[k])


In [10]:
common = pd.read_csv(r'../300w/cropped/face_landmarks_300w_valid_common_cropped.csv', index_col=0)
challenge = pd.read_csv(r'../300w/cropped/face_landmarks_300w_valid_challenge_cropped.csv', index_col=0)
valid  = common.append(challenge, ignore_index=True)
valid.to_csv(r'../300w/cropped/face_landmarks_300w_valid_cropped.csv')

In [7]:
datum

NameError: name 'datum' is not defined